## How to locally run parallel code with mpi4py in an IPython notebook:

The prerequisite for this is a working installation of some MPI distribution.

Using Ubuntu or some derivative, I recommend using OpenMPI which can be istalled from the repository by means of the following packages: *libopenmpi-dev, openmpi-bin, openmpi-doc*.

Now, you can already run MPI enabled code from you shell by calling

*$mpirun -n [numbmer_of_threads] python [script_to_run.py]*

To use MPI with iPython, one has to install ipyparallel:

via pip: *$pip install ipyparallel*

via conda: *$conda install ipyparallel*

and then enable the Clusters tab in ipython via

*$ ipcluster nbextension enable*

To make MPI acessable via mpi4py in an ipython notebook, one has to do the following:
open one shell and start the ipcontroller:

*$ipcontroller*

open another shell and start a number of engines:

*$mpirun -n [number of engines] ipengine --mpi=mpi4py*

and then connect to the engines via the following fragment of code:

In [147]:
%%px
import os


def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return root
path = find('02_LocalParallelization.ipynb', '/home/')
print(path)
os.chdir(path)

[stdout:0] /home/jakob/Project_PyMofa/tutorial
[stdout:1] /home/jakob/Project_PyMofa/tutorial
[stdout:2] /home/jakob/Project_PyMofa/tutorial


In [148]:
from ipyparallel import Client
c = Client()
view = c[:]
print(c.ids)

[0, 1, 2]


Now, to make the code run on all of our engines (and not just on one), the following cells have to start with the [__parallel magic__](https://ipython.org/ipython-doc/3/parallel/magics.html) command *%%px*

In [149]:
%%px
from mpi4py import MPI
com = MPI.COMM_WORLD
print(com.Get_rank())

[stdout:0] 2
[stdout:1] 1
[stdout:2] 0


Now, that we have MPI running, and mpi4py recognizing the nodes and their ranks, we can continue with the predator prey exercise, that we know from the first tutorial.

First, define the model:

In [150]:
%%px
import numpy as np
import matplotlib.pyplot as plt

def predprey_model(prey_birth_rate=0.1, prey_mortality=0.1, 
                   predator_efficiency=0.1, predator_death_rate=0.01,
                   initial_prey=1., initial_predators=1.,
                   time_length=1000):
    """Discrete predetor prey model."""
    A = -1 * np.ones(time_length)
    B = -1 * np.ones(time_length)
    A[0] = initial_prey
    B[0] = initial_predators
    for t in range(1, time_length):
        A[t] = A[t-1] + prey_birth_rate * A[t-1] - prey_mortality * B[t-1]*A[t-1]
        B[t] = B[t-1] + predator_efficiency * B[t-1]*A[t-1] - predator_death_rate * B[t-1] +\
            0.02 * (0.5 - np.random.rand())
    return A, B

#preys, predators = predprey_model()
#plt.plot(preys, label="preys") 
#plt.plot(predators, label="predators")
#plt.legend()
#plt.show()

Then import the experiment_handling class from pymofa and define a run function:

In [151]:
%%px
# imports
from pymofa.experiment_handling import experiment_handling as eh
import itertools as it
import pandas as pd
# import cPickle


#Definingh the experiment execution function
#      it gets paramater you want to investigate, plus `filename` as the last parameter
def RUN_FUNC(prey_birth_rate=0.1, prey_mortality=0.1, 
                   predator_efficiency=0.1, predator_death_rate=0.01,
                   initial_prey=1., initial_predators=1.,
                   time_length=1000, filename='./'):
    """Insightful docstring."""
    print(prey_birth_rate, prey_mortality, 
                   predator_efficiency, predator_death_rate,
                   initial_prey, initial_predators,
                   time_length)
    # one could also do more complicated stuff here, e.g. drawing something from a random distribution
    
    # running the model
    # TO DO: there seems to be a problem passing arguments to function
    #preys, predators = predprey_model(prey_birth_rate, prey_mortality, 
    #                                  predator_efficiency, predator_death_rate,
    #                                  initial_prey, initial_predators,
    #                                  time_length)
    preys, predators = predprey_model(
    )
    print(preys)
    # preparing the data
    res = pd.DataFrame({"preys": np.array(preys),
                        "predators": np.array(predators)})
    
    # Save Result
    res.to_pickle(filename)
    
    # determine exit status (if something went wrong)
    # if exit status > 0 == run passen
    # if exit status < 0 == Run Failed
    exit_status = 1
    
    # RUN_FUNC needs to return exit_status
    return exit_status

Specify the necessary parameters, generate their combinations and feed them to an experiment handle:

In [152]:
%%px
# Path where to Store the simulated Data
SAVE_PATH_RAW = "./dummy/pymofatutorial"

# Parameter combinations to investiage

prey_birth_rate = [0.1]
predator_death_rate = [0.1]
initial_pop = [1.]
            
PARAM_COMBS = list(it.product(prey_birth_rate, predator_death_rate, initial_pop))

# Sample Size
SAMPLE_SIZE = 5

# INDEX 
INDEX = {0: 'prey_birth_rate', 1: 'predator_death_rate', 2: 'initial_prey'}

# initiate handle instance with experiment variables
handle = eh(SAMPLE_SIZE, PARAM_COMBS, INDEX, SAVE_PATH_RAW)

And finally run the model - now in parallel:

In [153]:
%%px
# Compute experiemnts raw data
handle.compute(RUN_FUNC)

[stdout:0] 
(0.1, 0.1, 1.0, 0.01, 1.0, 1.0, 1000)
[  1.00000000e+00   1.00000000e+00   9.91669934e-01   9.73224371e-01
   9.44791004e-01   9.06819517e-01   8.58877059e-01   8.02138437e-01
   7.39747517e-01   6.72536995e-01   6.03743453e-01   5.35319353e-01
   4.68517671e-01   4.05063204e-01   3.46316644e-01   2.93350929e-01
   2.46817801e-01   2.06208785e-01   1.71283643e-01   1.41795735e-01
   1.16992016e-01   9.64107905e-02   7.94897163e-02   6.55505941e-02
   5.40807452e-02   4.47134058e-02   3.70181244e-02   3.06684858e-02
   2.54740187e-02   2.12261565e-02   1.77142128e-02   1.48330370e-02
   1.24584396e-02   1.04792033e-02   8.83686194e-03   7.46543242e-03
   6.31690934e-03   5.36268340e-03   4.56999541e-03   3.90972270e-03
   3.35329693e-03   2.88211167e-03   2.48536488e-03   2.15131097e-03
   1.86707609e-03   1.62340693e-03   1.41628462e-03   1.23834718e-03
   1.08523536e-03   9.54456339e-04   8.40787010e-04   7.42950405e-04
   6.58347759e-04   5.85263586e-04   5.20940939e-04  

And if everyting whent well, the calculations should have been splitted between all the engines that you've started in the beginning.

To run you experiments in scripts outside of an IPython notebook, simply run you experiment script (defining a run function, an experiment handle and calling the compute routine of that handle) with mpirun in a terminal 